In [1]:
import os

import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)

## Load the Data

Get file

In [2]:
with open("./01013500_lump_cida_forcing_leap.txt", 'r') as fp:
    # load area from header
    fp.readline()
    fp.readline()
    area = int(fp.readline())
    # load the dataframe from the rest of the stream
    df = pd.read_csv(fp, sep='\s+')
    df["date"] = pd.to_datetime(df.Year.map(str) + "/" + df.Mnth.map(str) + "/" + df.Day.map(str),
                                format="%Y/%m/%d")
    df = df.set_index("date")

df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa)
date,,,,,,,,,,,
1980-01-01,1980,1,1,12,30172.51,0.0,153.40,0.0,-6.54,-16.30,171.69
1980-01-02,1980,1,2,12,30253.10,0.0,145.27,0.0,-6.18,-15.22,185.94
1980-01-03,1980,1,3,12,30344.18,0.0,146.96,0.0,-9.89,-18.86,138.39
1980-01-04,1980,1,4,12,30408.33,0.0,146.20,0.0,-10.98,-19.76,120.06
1980-01-05,1980,1,5,12,30413.48,0.0,170.43,0.0,-11.29,-22.21,117.87


In [3]:
df['date'] = df.index
df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa),date
date,,,,,,,,,,,,
1980-01-01,1980,1,1,12,30172.51,0.0,153.40,0.0,-6.54,-16.30,171.69,1980-01-01
1980-01-02,1980,1,2,12,30253.10,0.0,145.27,0.0,-6.18,-15.22,185.94,1980-01-02
1980-01-03,1980,1,3,12,30344.18,0.0,146.96,0.0,-9.89,-18.86,138.39,1980-01-03
1980-01-04,1980,1,4,12,30408.33,0.0,146.20,0.0,-10.98,-19.76,120.06,1980-01-04
1980-01-05,1980,1,5,12,30413.48,0.0,170.43,0.0,-11.29,-22.21,117.87,1980-01-05


### **Compute mean daily precipitations**

In [4]:
len(df)

12784

In [5]:
df.rename(columns={"prcp(mm/day)": "prcp", "tmax(C)": "t_max", "tmin(C)": "t_min"}, inplace=True)
df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp,srad(W/m2),swe(mm),t_max,t_min,vp(Pa),date
date,,,,,,,,,,,,
1980-01-01,1980,1,1,12,30172.51,0.0,153.40,0.0,-6.54,-16.30,171.69,1980-01-01
1980-01-02,1980,1,2,12,30253.10,0.0,145.27,0.0,-6.18,-15.22,185.94,1980-01-02
1980-01-03,1980,1,3,12,30344.18,0.0,146.96,0.0,-9.89,-18.86,138.39,1980-01-03
1980-01-04,1980,1,4,12,30408.33,0.0,146.20,0.0,-10.98,-19.76,120.06,1980-01-04
1980-01-05,1980,1,5,12,30413.48,0.0,170.43,0.0,-11.29,-22.21,117.87,1980-01-05


In [6]:
df["t_mean"] = (df["t_max"] + df["t_min"]) / 2
df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp,srad(W/m2),swe(mm),t_max,t_min,vp(Pa),date,t_mean
date,,,,,,,,,,,,,
1980-01-01,1980,1,1,12,30172.51,0.0,153.40,0.0,-6.54,-16.30,171.69,1980-01-01,-11.420
1980-01-02,1980,1,2,12,30253.10,0.0,145.27,0.0,-6.18,-15.22,185.94,1980-01-02,-10.700
1980-01-03,1980,1,3,12,30344.18,0.0,146.96,0.0,-9.89,-18.86,138.39,1980-01-03,-14.375
1980-01-04,1980,1,4,12,30408.33,0.0,146.20,0.0,-10.98,-19.76,120.06,1980-01-04,-15.370
1980-01-05,1980,1,5,12,30413.48,0.0,170.43,0.0,-11.29,-22.21,117.87,1980-01-05,-16.750


In [7]:
from catchment_attribures import process_basin

df = process_basin(df, gauge_id="01013500")
df.head()

,gauge_id,p_mean,frac_snow,high_prec_freq,high_prec_dur,low_prec_freq,low_prec_dur
0,01013500,3.120536,0.274001,13.457143,1.328896,202.028571,3.393134


In [8]:
data = """gauge_id;p_mean;pet_mean;p_seasonality;frac_snow;aridity;high_prec_freq;high_prec_dur;high_prec_timing;low_prec_freq;low_prec_dur;low_prec_timing
01013500;3.12667898699521;1.97155451060917;0.187940258706929;0.313440357191799;0.63055865946247;12.95;1.34895833333333;son;202.2;3.4271186440678;mam"""

In [9]:
import io

df_base = pd.read_csv(io.StringIO(data), sep=";")
df_base = df_base[df.columns]
df_base.head()

,gauge_id,p_mean,frac_snow,high_prec_freq,high_prec_dur,low_prec_freq,low_prec_dur
0,1013500,3.126679,0.31344,12.95,1.348958,202.2,3.427119


In [15]:
cols_to_calc = ["p_mean", "frac_snow", "high_prec_freq", "high_prec_dur", "low_prec_freq", "low_prec_dur"]
df_err = (df[cols_to_calc] - df_base[cols_to_calc]) / df_base[cols_to_calc]

df_err["gauge_id"] = df.gauge_id
df_err = df_err[["gauge_id", *cols_to_calc]]
df_err.head()

,gauge_id,p_mean,frac_snow,high_prec_freq,high_prec_dur,low_prec_freq,low_prec_dur
0,01013500,-0.001965,-0.125828,0.039162,-0.014872,-0.000848,-0.009916


In [5]:
df = df[df["prcp(mm/day)"] > 0]
len(df)

7067

In [6]:
p_mean = df["prcp(mm/day)"].mean()
p_mean

5.644941276354889

In [7]:
p_mean_df = df.groupby('Year')["prcp(mm/day)"].mean()
p_mean_df

Year
1980    4.555885
1981    6.308341
1982    5.195794
1983    6.292556
1984    5.193649
1985    4.501157
1986    5.068811
1987    4.498146
1988    4.307130
1989    4.562326
1990    5.371416
1991    5.351478
1992    5.077778
1993    5.141652
1994    5.123165
1995    5.181171
1996    5.573081
1997    5.337784
1998    4.861644
1999    6.150323
2000    6.363027
2001    4.557910
2002    4.869850
2003    7.056322
2004    6.542096
2005    7.964660
2006    6.478404
2007    7.060533
2008    6.439031
2009    5.954607
2010    6.206995
2011    6.371173
2012    6.838137
2013    6.660173
2014    6.446167
Name: prcp(mm/day), dtype: float64

In [8]:
p_mean_df.mean()

5.698924840264767

### **Daily Fraction of Snow Precipitations** (mean temperature < 0)

In [9]:
df_snow_frac_daily = df[df["tmax(C)"] < 0]
df_snow_frac_daily = df_snow_frac_daily[(df_snow_frac_daily["prcp(mm/day)"].notna()) & (df_snow_frac_daily["prcp(mm/day)"] > 0)]
df_snow_frac_daily = df_snow_frac_daily["prcp(mm/day)"].sum() / df["prcp(mm/day)"].sum()
df_snow_frac_daily

0.17221628965627883

### **High Precipitation Frequency Days** (mean across years)

In [10]:
high_prec_freq_df = df[df["prcp(mm/day)"] >= 5 * p_mean]
high_prec_freq_df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa),date
date,,,,,,,,,,,,
1981-08-05,1981,8,5,12,52210.25,29.15,292.47,0.0,26.06,13.02,1482.62,1981-08-05
1981-08-06,1981,8,6,12,52184.88,55.48,162.40,0.0,18.54,12.39,1431.56,1981-08-06
1981-08-07,1981,8,7,12,51864.66,29.44,143.12,0.0,16.34,11.21,1336.80,1981-08-07
1981-08-17,1981,8,17,12,50112.03,39.31,169.99,0.0,17.86,11.87,1389.82,1981-08-17
1981-10-24,1981,10,24,12,36393.29,28.87,162.72,0.0,8.75,-0.66,587.55,1981-10-24


In [11]:
high_prec_freq_df["prcp(mm/day)"].mean()

35.03921875

In [12]:
high_prec_freq = high_prec_freq_df.groupby('Year').size()
print("Frequency of high precipitation days for each year:\n", high_prec_freq, "\n")
print("Average frequency of high precipitation days over all years: ", high_prec_freq.mean())

Frequency of high precipitation days for each year:
 Year
1981     5
1982     1
1983     2
1986     1
1989     1
1990     4
1991     2
1992     1
1995     1
1996     2
1997     1
1999     2
2000     1
2001     1
2002     1
2003     1
2004     3
2005    13
2006     1
2007     2
2008     3
2009     1
2010     4
2011     5
2012     3
2014     2
dtype: int64 

Average frequency of high precipitation days over all years:  2.4615384615384617


### **High Precipitation Duration**

Average duration of high precipitation events (number of consecutive days with >= 5 times mean daily precipitation)

In [13]:
s = high_prec_freq_df.groupby('Year')["date"].diff().dt.days.ne(1).cumsum()
high_prec_freq_df = high_prec_freq_df.groupby(['Year', s]).size().reset_index(level=1, drop=True)
high_prec_freq_df.values

array([3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 2, 2, 1, 2, 1, 1])

In [14]:
high_prec_freq_df_mean_year = high_prec_freq_df.groupby('Year').mean()
print("Average high precipitation duration for each year:\n", high_prec_freq_df_mean_year, "\n\n")
print(f"Average high precipitation duration overall: {high_prec_freq_df_mean_year.mean()}")

Average high precipitation duration for each year:
 Year
1981    1.666667
1982    1.000000
1983    1.000000
1986    1.000000
1989    1.000000
1990    1.000000
1991    1.000000
1992    1.000000
1995    1.000000
1996    1.000000
1997    1.000000
1999    1.000000
2000    1.000000
2001    1.000000
2002    1.000000
2003    1.000000
2004    1.500000
2005    1.300000
2006    1.000000
2007    1.000000
2008    1.000000
2009    1.000000
2010    1.333333
2011    1.666667
2012    1.500000
2014    1.000000
dtype: float64 


Average high precipitation duration overall: 1.1141025641025641


### **Low Precipitation Frequency** (< 1 mm/day)

In [15]:
low_prec_freq_df = df[df["prcp(mm/day)"] < 1]
low_prec_freq_df.head()

,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa),date
date,,,,,,,,,,,,
1980-01-14,1980,1,14,12,31209.29,0.06,196.58,0.0,-0.73,-12.80,237.02,1980-01-14
1980-02-02,1980,2,2,12,33868.83,0.83,128.42,0.0,-11.36,-16.25,169.24,1980-02-02
1980-02-04,1980,2,4,12,34214.42,0.69,165.52,0.0,-9.06,-15.25,186.32,1980-02-04
1980-02-05,1980,2,5,12,34319.71,0.35,218.07,0.0,-7.07,-15.47,193.90,1980-02-05
1980-02-29,1980,2,29,12,38893.10,0.01,369.39,0.0,-11.02,-24.82,80.00,1980-02-29


In [16]:
low_prec_freq = low_prec_freq_df.groupby(low_prec_freq_df.date.dt.year).size()
print("Frequency of low precipitation days for each year:\n", low_prec_freq, "\n")
print("Average frequency of low precipitation days over all years: ", low_prec_freq.mean())

Frequency of low precipitation days for each year:
 date
1980    52
1981    48
1982    46
1983    46
1984    50
1985    57
1986    57
1987    51
1988    61
1989    45
1990    42
1991    46
1992    45
1993    48
1994    46
1995    33
1996    52
1997    42
1998    47
1999    30
2000    27
2001    34
2002    42
2003    22
2004    18
2005    27
2006    23
2007    21
2008    42
2009    29
2010    32
2011    35
2012    23
2013    13
2014    22
dtype: int64 

Average frequency of low precipitation days over all years:  38.68571428571428


### **Low Precipitation Duration**

Average duration of dry days (number of consecutive days with precipitation < 1 mm/day)

In [17]:
low_prec_freq_df["year"] = pd.DatetimeIndex(low_prec_freq_df["date"]).year
low_prec_freq_df.head()

/tmp/ipykernel_64646/3575902681.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_prec_freq_df["year"] = pd.DatetimeIndex(low_prec_freq_df["date"]).year


,Year,Mnth,Day,Hr,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa),date,year
date,,,,,,,,,,,,,
1980-01-14,1980,1,14,12,31209.29,0.06,196.58,0.0,-0.73,-12.80,237.02,1980-01-14,1980
1980-02-02,1980,2,2,12,33868.83,0.83,128.42,0.0,-11.36,-16.25,169.24,1980-02-02,1980
1980-02-04,1980,2,4,12,34214.42,0.69,165.52,0.0,-9.06,-15.25,186.32,1980-02-04,1980
1980-02-05,1980,2,5,12,34319.71,0.35,218.07,0.0,-7.07,-15.47,193.90,1980-02-05,1980
1980-02-29,1980,2,29,12,38893.10,0.01,369.39,0.0,-11.02,-24.82,80.00,1980-02-29,1980


In [18]:
s = low_prec_freq_df.groupby('year').date.diff().dt.days.ne(1).cumsum()
low_prec_freq_df = low_prec_freq_df.groupby(['year', s]).size().reset_index(level=1, drop=True)
low_prec_freq_df

year
1980    1
1980    1
1980    2
1980    1
1980    1
1980    1
1980    1
1980    1
1980    2
1980    1
1980    2
1980    1
1980    1
1980    1
1980    2
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    2
1980    1
1980    2
1980    1
1980    2
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1980    1
1981    2
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    2
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    2
1981    1
1981    1
1981    1
1981    1
1981    1
1981    2
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1981    1
1982    1
1982    1
1982    1
1982    3
1982    1
1982    1
1982    1
1982    2
1982    1
1982    1
1982 

In [19]:
low_prec_freq_df_mean_year = low_prec_freq_df.groupby('year').mean()
print("Average low precipitation duration for each year:\n", low_prec_freq_df_mean_year, "\n\n")
print(f"Average low precipitation duration overall: {low_prec_freq_df_mean_year.mean()}")

Average low precipitation duration for each year:
 year
1980    1.155556
1981    1.090909
1982    1.121951
1983    1.095238
1984    1.219512
1985    1.266667
1986    1.239130
1987    1.159091
1988    1.418605
1989    1.250000
1990    1.135135
1991    1.069767
1992    1.216216
1993    1.200000
1994    1.150000
1995    1.064516
1996    1.238095
1997    1.135135
1998    1.146341
1999    1.000000
2000    1.080000
2001    1.133333
2002    1.105263
2003    1.100000
2004    1.058824
2005    1.038462
2006    1.210526
2007    1.050000
2008    1.235294
2009    1.035714
2010    1.066667
2011    1.060606
2012    1.210526
2013    1.000000
2014    1.157895
dtype: float64 


Average low precipitation duration overall: 1.1404278752881554
